In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/FR/faster_rcnn_classification/')
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Mounted at /content/drive
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


# Data split

In [ ]:
import os
import random
import numpy as np
import shutil
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from PIL import Image
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.datasets import ImageFolder
import time
import pandas as pd
from mobilenetv3.mobilenetv3 import mobilenetv3_large, mobilenetv3_small

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    # torch.backends.cudnn.deterministic = True  # type: ignore
    # torch.backends.cudnn.benchmark = True  # type: ignore
  
seed_everything()

net_large = mobilenetv3_large()
# net_small = mobilenetv3_small()

net_large.load_state_dict(torch.load('mobilenetv3/pretrained/mobilenetv3-large-1cd25616.pth'))
# net_small.load_state_dict(torch.load('mobilenetv3/pretrained/mobilenetv3-small-55df8e1f.pth'))

<All keys matched successfully>

In [ ]:
img = Image.open('./dataset_facial/dataset_facial_real/gs/0_0_gs_0.png')
img = img.convert('RGB')

default_aug = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_aug = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=20),
])

img = default_aug(img)
img = train_aug(img)
print(torch.sum(img))

tensor(24959.6133)


## Dataset

In [ ]:
class CustumDataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = data
        self.target = target
        self.transform = transform
    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        path = self.data[idx]
        img = Image.open(path)
        img = img.convert('RGB')

        if self.transform:
          img = self.transform(img)
        
        return img, int(self.target[idx])

In [ ]:
temp_dataset = ImageFolder(root='../../dataset_facial_cleaned//', transform = default_aug)
x_list = [temp_dataset.imgs[idx][0] for idx in range(len(temp_dataset.imgs))]
y_list = [temp_dataset.imgs[idx][1] for idx in range(len(temp_dataset.imgs))]

train_dataset = CustumDataset(x_list,y_list,transform = default_aug)
# loader_train = DataLoader(train_dataset, batch_size =32 , shuffle=True, num_workers=4)
# loader_valid = DataLoader(Dataset(X_val, y_val, None, False), batch_size =64 , shuffle=False, num_workers=4)
# loader_test = DataLoader(Dataset(X_test, y_test,None, False), batch_size =64 , shuffle=False, num_workers=4)

from sklearn.model_selection import KFold
splits=KFold(n_splits=5,shuffle=True,random_state=42)

### Model structure

In [ ]:
torchsummary.summary(net_large, (3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 64, 64]             432
       BatchNorm2d-2           [-1, 16, 64, 64]              32
             ReLU6-3           [-1, 16, 64, 64]               0
         h_sigmoid-4           [-1, 16, 64, 64]               0
           h_swish-5           [-1, 16, 64, 64]               0
            Conv2d-6           [-1, 16, 64, 64]             144
       BatchNorm2d-7           [-1, 16, 64, 64]              32
              ReLU-8           [-1, 16, 64, 64]               0
          Identity-9           [-1, 16, 64, 64]               0
           Conv2d-10           [-1, 16, 64, 64]             256
      BatchNorm2d-11           [-1, 16, 64, 64]              32
 InvertedResidual-12           [-1, 16, 64, 64]               0
           Conv2d-13           [-1, 64, 64, 64]           1,024
      BatchNorm2d-14           [-1, 64,

In [ ]:
import glob
list_name_full = glob.glob('../../dataset_facial/dataset_facial_unknown/*')
list_name = [name.split('/')[-1].split('.')[0] for name in list_name_full]
list_copy_name = []
list_name_target = os.listdir('../../dataset_facial_cleaned/dataset_facial_unknown/')
list_name_target = [name.split('.')[0] for name in list_name_target]
len(list_name)
for idx in range(len(list_name)):
  if list_name[idx] not in list_name_target:
    list_copy_name.append(list_name_full[idx])
    

In [ ]:
import shutil
target_root = '../../dataset_facial_cleaned_testset/dataset_facial_unknown/'
os.makedirs(target_root,exist_ok=True)

for idx in range(len(list_copy_name)):
  shutil.copy(list_copy_name[idx], target_root + list_name[idx] + '.png')

In [ ]:
import collections
collections.Counter(y_list)

Counter({0: 206, 1: 230})

In [ ]:
from torch.cuda.amp import GradScaler
from torch.cuda.amp import autocast
import copy
from torch.utils.data import random_split,SubsetRandomSampler
import torch.nn as nn
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 
net_large.to(device)

scaler = GradScaler()
num_epochs = 150
params = [p for p in net_large.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=1e-3, momentum=0.9, weight_decay=0.0005)

### Time cost

In [ ]:
import time
img = img.cuda().unsqueeze(0)
net_large.eval()
starttime = time.time()
with torch.no_grad():
  net_large(img)
print(time.time()-starttime)

0.013254404067993164


In [ ]:
import time
# img = img.cuda().unsqueeze(0)
img = img.to('cpu')
net_large.eval()
net_large.to('cpu')
starttime = time.time()
with torch.no_grad():
  net_large(img)
print(time.time()-starttime)

0.15786242485046387


# Train

In [ ]:
best_ep = 0; best_loss =10000000000; best_acc = 0;
transform = None

for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(train_dataset)))):
  model = copy.deepcopy(net_large).cuda()
  print('----------------------train start--------------------------')
  model.train(); model.cuda();
  final_model=None; best_ep = 0; best_loss =10000000000; best_acc = 0;
  loss_fn = nn.CrossEntropyLoss().cuda()
  params = [p for p in model.parameters() if p.requires_grad]
  optimizer = torch.optim.SGD(params, lr=0.005,
                                  momentum=0.9, weight_decay=0.0005)
  
  train_sampler = SubsetRandomSampler(train_idx)
  test_sampler = SubsetRandomSampler(val_idx)
  train_loader = DataLoader(train_dataset, batch_size=16, sampler=train_sampler)
  test_loader = DataLoader(train_dataset, batch_size=16, sampler=test_sampler)

  for epoch in range(num_epochs):
      start = time.time()
      model.train()
      for mode in ['Train','Valid']:
        epoch_loss = 0; epoch_acc = 0; epoch_all = 0;
        if mode == 'Train':
          loader = train_loader
          model.train()
        else:
          loader = test_loader
          model.eval()

        for xx, yy in loader:
          if mode=='Train':
            xx = train_aug(xx)
            # print(torch.sum(xx))
          xx = xx.to(device); yy = yy.to(device)
          len_acc = 0; len_all = 0;

          with autocast(enabled=True):
            if mode=='Train':
              y_pred = model(xx) 
              loss = loss_fn(y_pred, yy)  # loss 계산
            else :
              with torch.no_grad():
                y_pred = model(xx) 
                loss = loss_fn(y_pred, yy)  # loss 계산

            if mode == 'Train':
              optimizer.zero_grad()
              scaler.scale(loss).backward()
              scaler.step(optimizer)
              scaler.update()

            a = y_pred.argmax(1)
            len_acc = len(a[(a == yy)])
            len_all = len(yy)           

            epoch_loss += loss
            epoch_acc += len_acc
            epoch_all += len_all
              
        epoch_loss = epoch_loss / epoch_all
        epoch_acc = epoch_acc / epoch_all
        print(f'epoch : {epoch+1}, acc : {epoch_acc}, Loss : {epoch_loss}')

        if epoch_acc > best_acc and mode=='Valid':
          best_loss = epoch_loss
          best_acc = epoch_acc
          best_ep = epoch+1
          final_model = copy.deepcopy(model)
          torch.save(model.state_dict(),f'./weights/5fold/v3large_acc{epoch_acc}_fold{fold}.pth')
      print("------------------------------------------------------------------------")
      if epoch-best_ep > 10 : break

# EVAL

v1

In [ ]:

temp_dataset = ImageFolder(root='../../dataset_facial_cleaned//', transform=default_aug)
x_list = [temp_dataset.imgs[idx][0] for idx in range(len(temp_dataset.imgs))]
y_list = [temp_dataset.imgs[idx][1] for idx in range(len(temp_dataset.imgs))]

train_dataset = CustumDataset(x_list,y_list,transform = default_aug)
# loader_train = DataLoader(train_dataset, batch_size =32 , shuffle=True, num_workers=4)

loader_test = DataLoader(temp_dataset, batch_size =64 , shuffle=False, num_workers=4)
final_model = model
# final_model = torch.load('./weights/5fold/v3large_acc1.0_fold1.pth')
from sklearn.metrics import classification_report
loss_ce = torch.nn.CrossEntropyLoss()

trues, y_preds = [],[]
Acc,Loss = 0,0

final_model.eval(); final_model.cuda()
list_yy, list_predicted = [],[]
for _, (xx, yy) in enumerate(loader_test):  # loader에서 배치 단위로 데이터를 가져옴
    with torch.no_grad():
        with autocast(enabled=True):
            xx, yy= xx.cuda(), torch.tensor(yy,dtype=torch.long).cuda() # 데이터를 GPU로 보냄
            y_pred = final_model(xx)  # 모델이 결과 계산
            Loss += torch.sum(loss_ce(y_pred,yy)).item()
            _, predicted = torch.max(y_pred.cpu().detach(), 1)
            Acc += sum(predicted == yy.cpu().detach())

    y_preds.append(predicted.cpu().detach())
    trues.append(yy.cpu().detach())
summary(vgg, (3, 224, 224))

Acc = Acc/len(y_list) * 100
Loss = Loss/len(y_list)

print(f"Test Acc : {Acc} | Loss : {Loss}")
# To see the results more easier
import itertools
import numpy as np
trues = list(itertools.chain(*list(trues)))
y_preds = list(itertools.chain(*list(y_preds)))
trues = [i.item() for i in trues]
y_preds = [i.item() for i in y_preds]
result = classification_report(trues, y_preds, labels=None, target_names=None, sample_weight=None, digits=4, output_dict=False, zero_division='warn')
print(result)

pd.DataFrame(np.array([trues,y_preds]).T,columns=['true','pred']).to_csv(f'./fourlabel_halfcrop_ep33_results.csv',index=False)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Test Acc : 100.0 | Loss : 4.9043941677872674e-05
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       206
           1     1.0000    1.0000    1.0000       230

    accuracy                         1.0000       436
   macro avg     1.0000    1.0000    1.0000       436
weighted avg     1.0000    1.0000    1.0000       436



v2

In [ ]:
import itertools
import numpy as np

final_model = net_large
final_model.load_state_dict(torch.load('./weights/5fold/v3large_acc1.0_fold1.pth'))
temp_dataset = ImageFolder(root='../../dataset_facial_cleaned_testset/', transform=default_aug)
x_list = [temp_dataset.imgs[idx][0] for idx in range(len(temp_dataset.imgs))]
y_list = [temp_dataset.imgs[idx][1] for idx in range(len(temp_dataset.imgs))]

train_dataset = CustumDataset(x_list,y_list,transform = default_aug)
# loader_train = DataLoader(train_dataset, batch_size =32 , shuffle=True, num_workers=4)

loader_test = DataLoader(temp_dataset, batch_size =64 , shuffle=False, num_workers=4)
# final_model = torch.load('./weights/5fold/v3large_acc1.0_fold1.pth')
from sklearn.metrics import classification_report
loss_ce = torch.nn.CrossEntropyLoss()

trues, y_preds = [],[]
Acc,Loss = 0,0

final_model.eval(); final_model.cuda()
list_yy, list_predicted = [],[]
for _, (xx, yy) in enumerate(loader_test):  # loader에서 배치 단위로 데이터를 가져옴
    with torch.no_grad():
        with autocast(enabled=True):
            xx, yy= xx.cuda(), torch.tensor(yy,dtype=torch.long).cuda() # 데이터를 GPU로 보냄
            y_pred = final_model(xx)  # 모델이 결과 계산
            Loss += torch.sum(loss_ce(y_pred,yy)).item()
            _, predicted = torch.max(y_pred.cpu().detach(), 1)
            Acc += sum(predicted == yy.cpu().detach())

    y_preds.append(predicted.cpu().detach())
    trues.append(yy.cpu().detach())

Acc = Acc/len(y_list) * 100
Loss = Loss/len(y_list)

print(f"Test Acc : {Acc} | Loss : {Loss}")
# To see the results more easier
trues = list(itertools.chain(*list(trues)))
y_preds = list(itertools.chain(*list(y_preds)))
trues = [i.item() for i in trues]
y_preds = [i.item() for i in y_preds]
result = classification_report(trues, y_preds, labels=None, target_names=None, sample_weight=None, digits=4, output_dict=False, zero_division='warn')
print(result)

pd.DataFrame(np.array([trues,y_preds]).T,columns=['true','pred']).to_csv(f'./fourlabel_halfcrop_ep33_results.csv',index=False)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Test Acc : 99.9267349243164 | Loss : 3.517611896075554e-05
              precision    recall  f1-score   support

           0     0.8730    1.0000    0.9322        55
           1     1.0000    0.9993    0.9996     10864

    accuracy                         0.9993     10919
   macro avg     0.9365    0.9996    0.9659     10919
weighted avg     0.9994    0.9993    0.9993     10919

